## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-10-10-47-54 +0000,bbc,Israel protesters intensify pressure against p...,https://www.bbc.com/news/articles/c4ged13jjglo...
1,2025-08-10-10-30-00 +0000,wsj,Trump’s Tariffs Won’t Solve U.S. Chip-Making D...,https://www.wsj.com/tech/trumps-tariffs-wont-s...
2,2025-08-10-10-29-46 +0000,wapo,What parts of Ukraine does Russia control as T...,https://www.washingtonpost.com/world/2025/08/1...
3,2025-08-10-10-12-33 +0000,nypost,Endangered sea turtle named Elton released bac...,https://nypost.com/2025/08/10/us-news/endanger...
4,2025-08-10-10-00-28 +0000,nypost,Staten Island principal admits affair with you...,https://nypost.com/2025/08/10/us-news/nyc-prin...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2328/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,30
17,ukraine,17
242,putin,15
84,california,12
18,russia,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
144,2025-08-09-18-03-49 +0000,nypost,"Ukraine, European leaders counter Russian ceas...",https://nypost.com/2025/08/09/world-news/ukrai...,96
67,2025-08-10-07-00-39 +0000,nyt,"In a Trump-Putin Summit, Ukraine Fears Losing ...",https://www.nytimes.com/2025/08/10/us/politics...,79
32,2025-08-10-09-47-25 +0000,nyt,Ukraine and Europe Project United Front Ahead ...,https://www.nytimes.com/2025/08/10/world/europ...,78
155,2025-08-09-16-14-54 +0000,bbc,Putin gives Trump envoy award for CIA official...,https://www.bbc.com/news/articles/c04rd19e5epo...,75
78,2025-08-10-04-01-08 +0000,nyt,"With Ukraine in the Balance, Trump and Putin H...",https://www.nytimes.com/2025/08/10/world/europ...,73


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
144,96,2025-08-09-18-03-49 +0000,nypost,"Ukraine, European leaders counter Russian ceas...",https://nypost.com/2025/08/09/world-news/ukrai...
114,36,2025-08-09-22-47-00 +0000,wsj,Adriana Kugler hasn’t said why she is leaving ...,https://www.wsj.com/economy/central-banking/su...
127,36,2025-08-09-20-17-00 +0000,bbc,Atlanta police officer dies after shooting nea...,https://www.bbc.com/news/articles/cp3ekez0k2xo...
92,35,2025-08-10-01-00-00 +0000,wsj,President Trump has said Vice President JD Van...,https://www.wsj.com/politics/policy/who-is-the...
90,32,2025-08-10-01-31-15 +0000,nypost,"Aspiring National Guardsman, 19, killed by dru...",https://nypost.com/2025/08/09/us-news/aspiring...
137,28,2025-08-09-19-18-00 +0000,nyt,Zelensky Rejects Ceding Territory to Russia Af...,https://www.nytimes.com/2025/08/09/world/europ...
151,28,2025-08-09-16-40-45 +0000,nypost,Ambassador Mike Huckabee slams British PM Keir...,https://nypost.com/2025/08/09/us-news/ambassad...
149,27,2025-08-09-17-02-05 +0000,bbc,Teenager arrested after three shot in New York...,https://www.bbc.com/news/articles/c3eznd9npqko...
122,23,2025-08-09-21-07-54 +0000,bbc,Trump nominates ex-Fox News host Tammy Bruce a...,https://www.bbc.com/news/articles/cj9wv49xr0xo...
183,23,2025-08-09-12-00-00 +0000,wsj,Twelve Republicans who want the Epstein files ...,https://www.wsj.com/politics/policy/epstein-fi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
